In [8]:
#Import libraries 
import qutip as qt
import numpy as np
from numpy import linalg
import itertools
from qutip.qip.operations import swap
import pickle
import json 
import os
from multiprocessing import Pool



In [10]:
%run OpenSYK_functions.ipynb

In [27]:
# Parameters 

P = {"w": [1e-1,1,1e1],"gamma/w":[1,1e1,1e2], "w/g": np.array([5e-2, 1e-1, 5e-1, 1, 5, 1e1, 5e1, 1e2, 5e2]),\
     "N":[10,4,2], \
     "gamma/beta" : np.array([1e-6, 1e-4, 1e-3,1e-2,1e-1, 1, 1e1,1e2,1e3]),\
     "AverageNumber":350}

len_i= len(P["w"])
len_j= len(P["gamma/w"])
len_k= len(P["w/g"])
len_l= len(P["gamma/beta"])
#Evolution time
end= 2000
times = np.append(np.linspace(0,10,200),np.arange(10,end,1))
times.size

2190

## Series time evolution

In [28]:
#linear sampling
# sampling 
# w, w/Gamma, N value
i,j,m= 0,0,-1

for k in range(len_k):

  for l in range(len_l):
    
    Model = create_Model(i,j,k,l,m)
    data= { "counts_d":[], "purity":[], "entropy":[], "eigenvalues":[], "eigenvalues_L":[],\
       "entropy_infty":[],"purity_infty":[] }
    N = Model.N
    entropy_max= np.log(2**(N//2+1))
    purity_min = 1/2**(N//2+1)
    for s in range (P["AverageNumber"]):
        
      purity=[]
      entropy=[]
      #random variables   
      H, eig, lamb, coupl_c, coupl_cm, quasi_norms, f =  Model.Hamiltonian1
        
      # time evolution for the system
      result_d, rho,  eig_L = solve2(times, coupl_c, coupl_cm, f, N, H, quasi_norms)
      counts_d= result_d.expect
      rhot = rho.states
     
      for n in range(len(rhot)):
          # purity
          purity.append((rhot[n]**2).tr())
          #entropy
          entropy.append(qt.entropy_vn(rhot[n]))
            
      
      purity = np.array(purity)
      entropy = np.array(entropy)
      
    #scaled quantities
      entropy = entropy/entropy_max
      purity = purity/purity_min

    
      #save data
      data["counts_d"].append(counts_d)
      data["eigenvalues"].append(eig)
      data["eigenvalues_L"].append(eig_L)
      data["purity"].append(purity)
      data["entropy"].append(entropy)
      
    #averaged behavior
    av_entropy= np.mean(data["entropy"],axis=0)
    av_eig=  np.mean(data["eigenvalues"], axis=0)  
    av_purity= np.mean(data["purity"],axis=0)
  #steady state value of physical quantities
    S= S_infty(Model.beta,av_eig)
    Pty=  P_infty(Model.beta,av_eig)
    data["entropy_infty"].append(S)
    data["purity_infty"].append(Pty)
        
    print('N=%s, w=%.3f, gamma=%.3f, g=%.3f, beta=%.3f'%(Model.N, Model.w, Model.gamma,\
              Model.g, Model.beta)," process saved")
    name = 'DataBatchParallel/N_%s/w%.3f_w_g%.3f_'%(Model.N, Model.w, Model.w_g)+\
        'N%.1f_'%(N)+'gamma_w%.3f_gamma_beta%.6f'\
         %(Model.gamma_w, Model.gamma_beta)+'.pickle'
    # loss and gain included 
    name_lg = 'DataBatchParallel/N_%s/loss_gain/w%.3f_g%.3f_'%(Model.N, Model.w, Model.g)+\
        'N%.1f_'%(N)+'gamma%.3f_beta%.6f'\
         %(Model.gamma, Model.beta)+'.pickle'
    with open(name_lg,'wb') as handle:
      pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL) 



## Parallel time evolution

In [ ]:


##################################################
#Calculate the time evolution of the system

#################################################

if __name__ == '__main__': 
  process=[]
  with Pool(8) as p:
    i,j=0,0
    for k in range(len_k):
      for l in range(len_l):
        process.append(p.apply_async(solve_batch,(i,j,k,l)) )
    process = np.array(process)
    process= process.reshape(len_k, len_l)
   
    for n in range(len_k):
      for p in range(len_l):
        np.random.seed(int.from_bytes(os.urandom(4), byteorder='little'))
        process[n][p].get()
    

